In [ ]:
pip install catboost

In [5]:
import os # 찾고 싶은 경로에 어떤 파일이
import numpy as np # 선형대수 연산
import pandas as pd
import gc # garbage collector, 불필요한 메모리 낭비를 줄이기 위해서

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor # 다운 받아야 함

from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [6]:
os.listdir('./drive/MyDrive/machine learning/data')

['friend.csv',
 'new_friend.csv',
 'ventura.csv',
 'abalone.data',
 'new_friend_index_true.csv',
 'test.csv',
 'train.csv']

In [23]:
# Load Train and Test data
train_df = pd.read_csv('./drive/MyDrive/machine learning/data/train.csv')
test_df = pd.read_csv('./drive/MyDrive/machine learning/data/test.csv')

In [24]:
# check data size
print('train_df.shape: ', train_df.shape)
print('test_df.shape: ', test_df.shape) # test data 니까 taget 값이 없어서 column이 1 작음

train_df.shape:  (4459, 4993)
test_df.shape:  (49342, 4992)


In [25]:
train_df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,...,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,1300000.0,...,0,0,1100000.0,0,0,0,0.0,0.0,14800000,0.0,1200000.0,0.0,0.0,0,0,0,0,0.0,4000000,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0


In [26]:
# Train and Test Data info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [27]:
# Data Preprocessing
# 결측치 유무
# 쓸모 없는 값들도 사실 결측치인데 isnull로 파악이 안됨
# 데이터를 전반적으로 살펴봐야 함
train_df.isnull().sum().sum()

0

In [28]:
test_df.isnull().sum().sum()

0

In [29]:
# 모델 학습에 의미 없는 상수 컬럼 삭제

colsToRemove = []
for col in train_df.columns:
  if col!="ID" and col != 'target':
    if train_df[col].std() == 0:
      colsToRemove.append(col)
train_df.drop(colsToRemove, axis=1, inplace=True)
test_df.drop(colsToRemove, axis=1, inplace=True)

In [31]:
print("삭제된 컬럼 수: ", len(colsToRemove))
print("삭제된 컬럼 : ", colsToRemove)

삭제된 컬럼 수:  256
삭제된 컬럼 :  ['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', '1352ddae5', '2b601ad67', '6e42ff7c7', '22196a84c', '0e410eb3d', '992e6d1d3', '90a742107',

In [36]:
# 중복 컬럼 삭제
groups = train_df.columns.to_series().groupby(train_df.dtypes).groups

dup_col = []
for key, value in groups.items():
  df_col = train_df[value].columns
  df = train_df[value]
  l_df_col = len(df_col)

  for i in range(l_df_col):
    i_df = df.iloc[:,i].values
    for j in range(i+1, l_df_col):
      j_df = df.iloc[:,j].values
      if np.array_equal(i_df, j_df):
        dup_col.append(df_col[i])
train_df = train_df.drop(dup_col, axis=1)
test_df = test_df.drop(dup_col, axis=1)

In [37]:
train_df.shape

(4459, 4732)

In [38]:
# Prepare data for Traing & Test
x_train = train_df.drop(['ID', 'target'], axis=1)
y_train = np.log1p(train_df['target'].values)

x_test = test_df.drop(['ID'],axis=1)
# train_test_split
dev_x, val_x, dev_y, val_y = train_test_split(x_train, y_train, test_size=0.2, random_state=2021)

In [44]:
# Build LightGBM Model & Predict

params = {
    'objective':'regression',
    'metric' : 'rmse',
    'num_leaves' : 40,
    'learning_rate':0.004,
    'bagging_fraction' : 0.6,
    'feature_fraction':0.6,
    'bagging_seed':2021,
    'verbosity':-1,
    'seed':2021
}

lgtrain = lgb.Dataset(dev_x, dev_y)
lgval = lgb.Dataset(val_x,val_y)

evals_result = {}
model = lgb.train(params=params,
                  train_set = lgtrain,
                  num_boost_round = 5000,
                  valid_sets = [lgtrain, lgval],
                  early_stopping_rounds = 100,
                  verbose_eval = 150,
                  evals_result = evals_result)

Training until validation scores don't improve for 100 rounds.
[150]	training's rmse: 1.5053	valid_1's rmse: 1.5553
[300]	training's rmse: 1.34576	valid_1's rmse: 1.47217
[450]	training's rmse: 1.23447	valid_1's rmse: 1.43298
[600]	training's rmse: 1.15071	valid_1's rmse: 1.4148
[750]	training's rmse: 1.08428	valid_1's rmse: 1.40695
[900]	training's rmse: 1.03057	valid_1's rmse: 1.40509
[1050]	training's rmse: 0.986511	valid_1's rmse: 1.40492
Early stopping, best iteration is:
[972]	training's rmse: 1.00853	valid_1's rmse: 1.40452


In [45]:
# np.log1p를 다시 되돌리는 exp => np.expm1
prediction = model.predict(x_test, num_iteration=model.best_iteration)
pred_test_y = np.expm1(prediction)

In [46]:
pred_test_y

array([2001453.0320639 , 1701792.84486827, 1546757.21256508, ...,
        888222.92595353,  501018.104024  , 1939020.78174481])

In [47]:
# feature importance
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({
    'feature':model.feature_name(),
    'split':model.feature_importance('split'),
    'gain':100*gain/gain.sum()
}).sort_values('gain',ascending=False)

In [48]:
featureimp

,feature,split,gain
4130,f190486d6,882,9.849580
2375,58e2e02e6,849,5.811981
4020,15ace8c9f,512,3.433981
3465,eeb9cd3aa,581,2.989733
2614,9fd594eec,355,2.526893
...,...,...,...
1852,6cf2d8705,0,0.000000
1855,09f827f1c,0,0.000000
1857,4ca7f1312,0,0.000000
1858,ad566b17c,0,0.000000
